In [1]:
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels
import hvplot.xarray

In [2]:
station = "Papa"

## Load DATA


### CMEMS


In [3]:
data_cmems = xr.open_dataset(f"../data/1_products/{station}_cmems.zarr", engine="zarr").load()
data_cmems

<xarray.Dataset> Size: 165MB
Dimensions:              (time: 8280, depth: 3, latitude: 12, longitude: 26)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 96B 46.5 47.5 48.5 ... 56.5 57.5
  * longitude            (longitude) float64 208B -156.5 -153.5 ... -128.5
  * time                 (time) datetime64[ns] 66kB 1998-01-01 ... 2020-09-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 62MB 6.87...
    npp                  (time, latitude, longitude) float64 21MB 191.1 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 62MB 90.5...
    zooc                 (time, latitude, longitude) float64 21MB 2.848 ... nan
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Climato


In [4]:
for i in range(1, 13):
    print(
        f"Month {i} : ",
        set(
            pd.DataFrame({"month": data_cmems.indexes["time"].month, "day": data_cmems.indexes["time"].day})
            .query(f"month == {i}")["day"]
            .unique()
        ),
    )

Month 1 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 2 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
Month 3 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 4 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 5 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 6 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 7 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 8 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


In [5]:
data_cmems_climato = data_cmems.groupby("time.dayofyear").mean()
data_cmems_climato

<xarray.Dataset> Size: 7MB
Dimensions:              (dayofyear: 366, depth: 3, latitude: 12, longitude: 26)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 96B 46.5 47.5 48.5 ... 56.5 57.5
  * longitude            (longitude) float64 208B -156.5 -153.5 ... -128.5
  * dayofyear            (dayofyear) int64 3kB 1 2 3 4 5 ... 362 363 364 365 366
Data variables:
    T                    (dayofyear, depth, latitude, longitude) float64 3MB ...
    npp                  (dayofyear, latitude, longitude) float64 914kB 205.0...
    pelagic_layer_depth  (dayofyear, depth, latitude, longitude) float64 3MB ...
    zooc                 (dayofyear, latitude, longitude) float64 914kB 2.042...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Expand to original time series interval


In [6]:
all_years = []
for year in range(int(data_cmems.time.min().dt.year), int(data_cmems.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = data_cmems_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
data_cmems_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
data_cmems_climato_expanded.time.attrs = data_cmems.time.attrs
data_cmems_climato_expanded

<xarray.Dataset> Size: 168MB
Dimensions:              (time: 8401, depth: 3, latitude: 12, longitude: 26)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 96B 46.5 47.5 48.5 ... 56.5 57.5
  * longitude            (longitude) float64 208B -156.5 -153.5 ... -128.5
  * time                 (time) datetime64[ns] 67kB 1998-01-01 ... 2020-12-31
Data variables:
    T                    (time, depth, latitude, longitude) float64 63MB 7.04...
    npp                  (time, latitude, longitude) float64 21MB 205.0 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 63MB 92.6...
    zooc                 (time, latitude, longitude) float64 21MB 2.042 ... nan
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Plot


In [ ]:
data_cmems_climato_expanded.mean(["latitude", "longitude"]).hvplot.line(
    x="time",
    groupby="depth",
    title=f"Climato {station} - CMEMS T",
)

## Export


In [9]:
data_cmems_climato_expanded.to_zarr(
    f"../data/1_products/{station}_cmems_climato.zarr",
    mode="w",
)